## 1.1. Продвинутые концепции Python

### 1.1.1. Декораторы и их применение

**Декоратор** - это функция, которая принимает другую функцию в качестве аргумента, добавляет к ней некоторую функциональность и возвращает новую функцию

Создание простого декоратора

In [5]:
# декоратор, который будет выводить сообщение до и после вызова функции.
def logger(func):
    def wrapper(*args, **kwargs):
        print(f"Вызов функции {func.__name__} с аргументами {args} и {kwargs}")
        result = func(*args, **kwargs)
        print(f"Функция {func.__name__} завершила выполнение")
        return result
    return wrapper

@logger
def add(a, b):
    return a + b

print(add(2, 3))

Вызов функции add с аргументами (2, 3) и {}
Функция add завершила выполнение
5


Создание декоратора с аргументами

In [8]:
# декоратор, который повторяет выполнение функции заданное количество раз
def repeat(num_times):
    def decorator(func):
        def wrapper(*args, **kwargs):
            for _ in range(num_times):
                result = func(*args, **kwargs)
            return result
        return wrapper
    return decorator

@repeat(3)
def greet(name):
    print(f"Привет, {name}!")

greet("Alex")

Привет, Alex!
Привет, Alex!
Привет, Alex!


Встроенные декораторы. @staticmethod и @classmethod и @property  
- @staticmethod используется для создания методов, которые не требуют доступа к экземпляру или классу.
- @classmethod используется для создания методов, которые работают с классом, а не с экземпляром.
- @property позволяет превратить метод в атрибут, который можно читать без вызова.

Декораторы можно применять последовательно. Порядок применения важен: декораторы выполняются снизу вверх.

#### Практика

In [13]:
# декоратор, который измеряет время выполнения функции и выводит его
import time

def timer(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"Функция {func.__name__} выполнилась за {end_time - start_time:.4f} секунд")
        return result
    return wrapper

@timer
def slow_function(sec):
    time.sleep(sec)

slow_function(3)

Функция slow_function выполнилась за 3.0009 секунд


In [14]:
# декоратор, который кэширует результаты функции, чтобы избежать повторных вычислений
def cache(func):
    cached_results = {}
    def wrapper(*args):
        if args in cached_results:
            print("Результат из кэша")
            return cached_results[args]
        result = func(*args)
        cached_results[args] = result
        print("Результат вычислен")
        return result
    return wrapper

@cache
def fibonacci(n):
    if n <= 1:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)

print(fibonacci(2))

Результат вычислен
Результат вычислен
Результат вычислен
1


In [15]:
# Декоратор для логирования аргументов и результата функции в файл
def log_to_file(filename):
    def decorator(func):
        def wrapper(*args, **kwargs):
            with open(filename, 'a') as f:
                f.write(f"Вызов функции {func.__name__} с аргументами: {args}, {kwargs}\n")
            result = func(*args, **kwargs)
            with open(filename, 'a') as f:
                f.write(f"Результат функции {func.__name__}: {result}\n")
            return result
        return wrapper
    return decorator

@log_to_file('log.txt')
def add(a, b):
    return a + b

add(2, 3)

5

In [16]:
# Декоратор для проверки, что аргументы функции являются числами
def check_numeric(func):
    def wrapper(*args, **kwargs):
        # Проверка позиционных аргументов (args)
        for arg in args:
            if not isinstance(arg, (int, float)):
                raise ValueError(f"Аргумент {arg} не является числом")

        # Проверка именованных аргументов (kwargs)
        for key, value in kwargs.items():
            if not isinstance(value, (int, float)):
                raise ValueError(f"Аргумент {key}={value} не является числом")

        # Если все аргументы прошли проверку, вызываем исходную функцию
        return func(*args, **kwargs)
    return wrapper

@check_numeric
def multiply(a, b):
    return a * b

print(multiply(2, 3))  # Работает
print(multiply(2, '3'))  # Вызовет исключение

6


ValueError: Аргумент 3 не является числом

In [ ]:
# Декоратор для ограничения количества вызовов функции
def limit_calls(max_calls):
    def decorator(func):
        def wrapper(*args, **kwargs):
            if wrapper.calls >= max_calls:
                raise Exception(f"Превышено максимальное количество вызовов функции {func.__name__}")
            wrapper.calls += 1
            return func(*args, **kwargs)
        wrapper.calls = 0
        return wrapper
    return decorator

@limit_calls(2)
def say_hello(name):
    print("Hello", name)

say_hello("Vasay1")
say_hello("Vasay2")
say_hello("Vasay3")

In [ ]:
@log_to_file('log.txt')
@check_numeric
@limit_calls(3)
def add(a, b):
    return a + b

add(2, 3)
add(4, 5)
add(6, 7)
add(8, 9)  # Вызовет исключение из-за ограничения на количество вызовов

### 1.1.2. Генераторы и итераторы

Генераторы и итераторы — это мощные инструменты Python для работы с последовательностями данных. Они позволяют эффективно работать с большими объемами данных, не загружая их полностью в память.

Итератор - это объект, который реализует протокол итерации.  
- \_\_iter__() - возвращает сам объект итератора.
- \_\_next__() - возвращает следующий элемент последовательности. Если элементы закончились, вызывается исключение StopIteration.

In [ ]:
# простой итератор, который возвращает числа от 0 до заданного предела
class MyIterator:
    def __init__(self, limit):
        self.limit = limit
        self.current = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.current < self.limit:
            self.current += 1
            return self.current - 1
        else:
            raise StopIteration

my_iter = MyIterator(3)
for num in my_iter:
    print(num)

Генератор — это функция, которая возвращает итератор. Вместо return используется ключевое слово yield, которое приостанавливает выполнение функции и возвращает значение. При следующем вызове функция продолжает выполнение с того места, где она была приостановлена.

In [18]:
# генератор, который возвращает числа от 0 до заданного предела
def my_generator(limit):
    current = 0
    while current < limit:
        yield current
        current += 1

gen = my_generator(3)
for num in gen:
    print(num)

0
1
2


In [20]:
# генератор, который возвращает квадраты чисел.
squares = (x * x for x in range(5))

for square in squares:
    print(square, end=" ")

0 1 4 9 16 

#### Практика

In [23]:
# генератор, который возвращает числа Фибоначчи до заданного предела
def fibonacci_generator(limit):
    a, b = 0, 1
    while a < limit:
        yield a
        a, b = b, a + b

for num in fibonacci_generator(200):
    print(num, end=" ")

0 1 1 2 3 5 8 13 21 34 55 89 144 

In [25]:
# итератор, который возвращает элементы списка в обратном порядке
class ReverseIterator:
    def __init__(self, data):
        self.data = data
        self.index = len(data)

    def __iter__(self):
        return self

    def __next__(self):
        if self.index > 0:
            self.index -= 1
            return self.data[self.index]
        else:
            raise StopIteration

# Использование итератора
my_list = [1, 2, 3, 4, 5]
rev_iter = ReverseIterator(my_list)
for item in rev_iter:
    print(item)

5
4
3
2
1


In [27]:
# генератор, который возвращает бесконечную последовательность чисел, начиная с заданного числа
def infinite_sequence(start):
    current = start
    while True:
        yield current
        current += 1

# Использование генератора (осторожно: бесконечный цикл!)
gen = infinite_sequence(10)
for _ in range(5):
    print(next(gen))

10
11
12
13
14


In [29]:
# генератор простых чисел до заданного предела
def primes(limit):
    if limit < 0:
        # Если предел отрицательный, функция завершает работу, так как простые числа определены только для натуральных чисел.
        return
    # Создается список sieve, где каждый элемент изначально равен True.
    # Элементы с индексами 0 и 1 устанавливаются в False, так как 0 и 1 не являются простыми числами.
    sieve = [True] * (limit + 1)
    sieve[0] = sieve[1] = False
    for num, is_prime in enumerate(sieve):
        if is_prime:
            yield num
            for multiple in range(num * num, limit + 1, num):
                sieve[multiple] = False

# Пример использования:
for prime in primes(10):
    print(prime)

2
3
5
7


In [31]:
# Итератор для плоского представления вложенного списка
def flatten_list(nested_list):
    for element in nested_list:
        if isinstance(element, list):
            yield from flatten_list(element)
        else:
            yield element

# Пример использования:
nested_list = [[1, 2], [3, 4], [5, [6, 7]]]
for item in flatten_list(nested_list):
    print(item, end=" ")

1 2 3 4 5 6 7 

### 1.1.3. Контекстные менеджеры (with)

Контекстные менеджеры в Python - это удобный способ управления ресурсами, такие как файлы, сетевые соединения или блокировки. Они гарантируют что ресурсы будут корректно освобождены, даже если в процессе работы возникает исключение.

Контекстный менеджер - это объект, который определяет методы \_\_enter__() и \_\_exit__(). Эти методы вызываются при входе в блок with и при выходе из него.

In [38]:
# создания контекстного менеджера
class MyContextManager:
    def __enter__(self):
        print("Вход в контекст")
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        print("Выход из контекста")
        if exc_type is not None:
            print(f"Произошло исключение: {exc_value}")
        return True  # Подавляем исключение

# Использование контекстного менеджера
with MyContextManager() as manager:
    print("Внутри контекста")
    raise ValueError("Ошибка!")

Вход в контекст
Внутри контекста
Выход из контекста
Произошло исключение: Ошибка!


In [40]:
# contextlib предоставляет декоратор @contextmanager, который упрощает создание контекстных менеджеров.
from contextlib import contextmanager

@contextmanager
def my_context_manager():
    print("Вход в контекст")
    try:
        yield
    except Exception as e:
        print(f"Произошло исключение: {e}")
    finally:
        print("Выход из контекста")

# Использование контекстного менеджера
with my_context_manager():
    print("Внутри контекста")
    raise ValueError("Ошибка!")

Вход в контекст
Внутри контекста
Произошло исключение: Ошибка!
Выход из контекста


#### Практика

In [43]:
# Управление временем выполнения
import time

import time

class Timer:
    def __enter__(self):
        self.start_time = time.time()
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self.end_time = time.time()
        print(f"Время выполнения: {self.end_time - self.start_time:.4f} секунд")

# Использование контекстного менеджера
with Timer():
    time.sleep(2)

Время выполнения: 2.0005 секунд


In [44]:
# Контекстный менеджер может управлять сетевыми соединениями, гарантируя их закрытие.
import socket

class SocketConnection:
    def __init__(self, host, port):
        self.host = host
        self.port = port

    def __enter__(self):
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.sock.connect((self.host, self.port))
        return self.sock

    def __exit__(self, exc_type, exc_value, traceback):
        self.sock.close()

# Использование контекстного менеджера
with SocketConnection('example.com', 80) as sock:
    sock.sendall(b'GET / HTTP/1.1\r\nHost: example.com\r\n\r\n')
    response = sock.recv(4096)
    print(response.decode())

HTTP/1.1 200 OK
Content-Type: text/html
ETag: "84238dfc8092e5d9c0dac8ef93371a07:1736799080.121134"
Last-Modified: Mon, 13 Jan 2025 20:11:20 GMT
Cache-Control: max-age=564
Date: Thu, 16 Jan 2025 18:56:35 GMT
Content-Length: 1256
Connection: keep-alive

<!doctype html>
<html>
<head>
    <title>Example Domain</title>

    <meta charset="utf-8" />
    <meta http-equiv="Content-type" content="text/html; charset=utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1" />
    <style type="text/css">
    body {
        background-color: #f0f0f2;
        margin: 0;
        padding: 0;
        font-family: -apple-system, system-ui, BlinkMacSystemFont, "Segoe UI", "Open Sans", "Helvetica Neue", Helvetica, Arial, sans-serif;
        
    }
    div {
        width: 600px;
        margin: 5em auto;
        padding: 2em;
        background-color: #fdfdff;
        border-radius: 0.5em;
        box-shadow: 2px 3px 7px 2px rgba(0,0,0,0.02);
    }
    a:link, a:visited {
        

In [45]:
# Контекстный менеджер для временного изменения директории
import os
from contextlib import contextmanager

@contextmanager
def change_dir(path):
    original_dir = os.getcwd()
    try:
        os.chdir(path)
        yield
    finally:
        os.chdir(original_dir)

# Использование контекстного менеджера
with change_dir('C:\\Users\\Anatolii\\Python'):
    print(f"Текущая директория: {os.getcwd()}")
print(f"Директория восстановлена: {os.getcwd()}")

Текущая директория: C:\Users\Anatolii\Python
Директория восстановлена: C:\Users\Anatolii\Python\1_Python_и_ООП


In [46]:
# Контекстный менеджер для работы с базой данных
import sqlite3

class DatabaseConnection:
    def __init__(self, db_name):
        self.db_name = db_name

    def __enter__(self):
        self.conn = sqlite3.connect(self.db_name)
        return self.conn

    def __exit__(self, exc_type, exc_value, traceback):
        self.conn.close()

# Использование контекстного менеджера
with DatabaseConnection('example.db') as conn:
    cursor = conn.cursor()
    cursor.execute('CREATE TABLE IF NOT EXISTS test (id INTEGER PRIMARY KEY, name TEXT)')
    conn.commit()

In [52]:
# контекстный менеджер, который временно изменяет текущую рабочую директорию, а затем возвращает её обратно
import os
from contextlib import contextmanager

@contextmanager
def change_dir(path):
    original_dir = os.getcwd()
    try:
        os.chdir(path)
        yield
    finally:
        os.chdir(original_dir)

# Использование контекстного менеджера
with change_dir('C:\\Users'):
    print(f"Текущая директория: {os.getcwd()}")
print(f"Директория восстановлена: {os.getcwd()}")

Текущая директория: C:\Users
Директория восстановлена: C:\Users\Anatolii\Python\1_Python_и_ООП


In [54]:
# контекстный менеджер, который логирует начало и конец выполнения блока кода
class Logger:
    def __enter__(self):
        print("Начало выполнения блока")
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        print("Конец выполнения блока")

# Использование контекстного менеджера
with Logger():
    print("Внутри блока")

Начало выполнения блока
Внутри блока
Конец выполнения блока


In [56]:
# контекстный менеджер, который управляет соединением с базой данных (например, SQLite).
import sqlite3

class DatabaseConnection:
    def __init__(self, db_name):
        self.db_name = db_name

    def __enter__(self):
        self.conn = sqlite3.connect(self.db_name)
        return self.conn

    def __exit__(self, exc_type, exc_value, traceback):
        self.conn.close()

# Использование контекстного менеджера
with DatabaseConnection('example.db') as conn:
    cursor = conn.cursor()
    cursor.execute('CREATE TABLE IF NOT EXISTS test (id INTEGER PRIMARY KEY, name TEXT)')
    conn.commit()

### 1.1.4. Работа с исключениями и кастомные исключения

Исключения в Python - это механизм обработки ошибок, который позволяет программе корректно реагировать на неожиданные ситуации.  
Исключение - это событие которое возникает во время выполнения программы и нарушает нормальный поток выполнения.

Для обработки исключений используются блоки:

- try — код, который может вызвать исключение.  
- except — блок, который выполняется, если исключение произошло.  
- finally — блок, который выполняется всегда, независимо от того, было исключение или нет.  

Встроенные исключения:
- ZeroDivisionError — деление на ноль.
- FileNotFoundError — файл не найден.
- TypeError — операция применена к объекту несоответствующего типа.
- ValueError — функция получает аргумент правильного типа, но некорректного значения.

In [13]:
# Пример обработки нескольких исключений
try:
    num = int(input("Введите число: "))
    result = 10 / num
    print(f"Результат: {result}")
except ValueError:
    print("Ошибка: введено не число")
except ZeroDivisionError:
    print("Ошибка: деление на ноль")

Введите число:  d


Ошибка: введено не число


Кастомные исключения позволяют создавать более понятные и специфичные ошибки для вашего приложения. Кастомные исключения создаются путем наследования от встроенного класса Exception или его подклассов.

Создание кастомных исключений. Кастомные исключения создаются путем наследования от встреонного класса EXception или его подклассов.

In [6]:
class MyCustomError(Exception):
    """Мое кастомное исключение."""
    def __init__(self, message):
        super().__init__(message)
        self.message = message

def check_value(value):
    if value < 0:
        raise MyCustomError("Значение не может быть отрицательным")

try:
    check_value(-10)
except MyCustomError as e:
    print(f"Ошибка: {e.message}")

Ошибка: Значение не может быть отрицательным


In [8]:
# Кастомные исключения могут быть частью иерархии, что позволяет группировать связанные ошибки.
class ValidationError(Exception):
    """Базовое исключение для ошибок валидации."""
    pass

class NegativeValueError(ValidationError):
    """Ошибка, если значение отрицательное."""
    pass

class TooLargeValueError(ValidationError):
    """Ошибка, если значение слишком большое."""
    pass

def validate_value(value):
    if value < 0:
        raise NegativeValueError("Значение не может быть отрицательным")
    if value > 100:
        raise TooLargeValueError("Значение не может быть больше 100")

try:
    validate_value(150)
except ValidationError as e:
    print(f"Ошибка валидации: {e}")    

Ошибка валидации: Значение не может быть больше 100


In [12]:
# Исключения часто используются для обработки ошибок ввода-вывода, сетевых запросов, работы с базами данных и т.д.
import requests

def fetch_data(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Проверка на ошибки HTTP
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Ошибка при запросе: {e}")
        return None

data = fetch_data("https://api.example.com/data")
if data:
    print(data)

Ошибка при запросе: HTTPSConnectionPool(host='api.example.com', port=443): Max retries exceeded with url: /data (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000183C9799CD0>: Failed to resolve 'api.example.com' ([Errno 11001] getaddrinfo failed)"))


Блок finally полезен для освобождения ресурсов (например, закрытия файлов или соединений с базой данных), даже если произошло исключение.

In [21]:
try:
    file = open("example.txt", "r")
    content = file.read()
    print(content)
except FileNotFoundError:
    print("Файл не найден")
finally:
    file.close()

Файл не найден


NameError: name 'file' is not defined

In [26]:
# Кастомное исключение для валидации email
class InvalidEmailError(Exception):
    """Ошибка, если email невалиден."""
    pass

def validate_email(email):
    if '@' not in email:
        raise InvalidEmailError("Email должен содержать символ @")

try:
    validate_email("user.example.com")
except InvalidEmailError as e:
    print(f"Ошибка: {e}")

Ошибка: Email должен содержать символ @


In [28]:
# Обработка исключений при работе с файлами
def read_file(file_path):
    try:
        with open(file_path, 'r') as file:
            return file.read()
    except FileNotFoundError:
        return "Ошибка: файл не найден"
    except PermissionError:
        return "Ошибка: нет доступа к файлу"

print(read_file("nonexistent.txt"))

Ошибка: файл не найден


In [30]:
# Иерархия исключений для калькулятор
class CalculatorError(Exception):
    """Базовое исключение для калькулятора."""
    pass
    
class DivisionByZeroError(CalculatorError):
    """Ошибка деления на ноль."""
    pass

class NegativeResultError(CalculatorError):
    """Ошибка, если результат отрицательный."""
    pass

def divide(a, b):
    if b == 0:
        raise DivisionByZeroError("Деление на ноль невозможно")
    result = a / b
    if result < 0:
        raise NegativeResultError("Результат отрицательный")
    return result

try:
    print(divide(10, -2))
except CalculatorError as e:
    print(f"Ошибка калькулятора: {e}")

Ошибка калькулятора: Результат отрицательный


In [32]:
# Кастомное исключение InvalidPasswordError и функция для проверки пароля
class InvalidPasswordError(Exception):
    """Исключение, которое выбрасывается, если пароль не соответствует требованиям."""
    pass

def validate_password(password):
    """Проверяет, что пароль содержит не менее 8 символов."""
    if len(password) < 8:
        raise InvalidPasswordError("Пароль должен содержать не менее 8 символов")
    return True

# Пример использования
try:
    validate_password("1234567")
except InvalidPasswordError as e:
    print(e)  # Выведет: Пароль должен содержать не менее 8 символов

Пароль должен содержать не менее 8 символов


In [34]:
# Функция для чтения JSON-файла с обработкой исключений
import json

def read_json_file(file_path):
    """Читает JSON-файл и обрабатывает возможные исключения."""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            return data
    except FileNotFoundError:
        print(f"Ошибка: Файл {file_path} не найден.")
    except json.JSONDecodeError:
        print(f"Ошибка: Файл {file_path} содержит некорректный JSON.")
    except Exception as e:
        print(f"Произошла неизвестная ошибка: {e}")

# Пример использования
data = read_json_file("example.json")
if data:
    print(data)

Ошибка: Файл example.json не найден.


In [36]:
# Иерархия исключений для работы с банковским счетом
class BankAccountError(Exception):
    """Базовое исключение для всех ошибок, связанных с банковским счетом."""
    pass

class InsufficientFundsError(BankAccountError):
    """Исключение, которое выбрасывается, если на счете недостаточно средств."""
    pass

class InvalidTransactionError(BankAccountError):
    """Исключение, которое выбрасывается, если транзакция некорректна."""
    pass

class BankAccount:
    def __init__(self, balance):
        self.balance = balance

    def withdraw(self, amount):
        if amount > self.balance:
            raise InsufficientFundsError("Недостаточно средств на счете")
        if amount <= 0:
            raise InvalidTransactionError("Сумма для снятия должна быть положительной")
        self.balance -= amount
        return self.balance

    def deposit(self, amount):
        if amount <= 0:
            raise InvalidTransactionError("Сумма для пополнения должна быть положительной")
        self.balance += amount
        return self.balance

# Пример использования
account = BankAccount(100)

try:
    account.withdraw(150)
except InsufficientFundsError as e:
    print(e)  # Выведет: Недостаточно средств на счете

try:
    account.deposit(-50)
except InvalidTransactionError as e:
    print(e)  # Выведет: Сумма для пополнения должна быть положительной

Недостаточно средств на счете
Сумма для пополнения должна быть положительной


### 1.1.5. Модуль collections (defaultdict, Counter, namedtuple)

Модуль **collections** в Python предоставляет специализированные типы данных, которые расширяют возможности стандартных структур, таких как списки, словари и кортежи. В этом занятии мы разберем три наиболее полезных класса из этого модуля: **defaultdict**, **Counter** и **namedtuple**.

**defaultdict** - это подкласс словаря, которы автоматически создает значения для несуществующих ключей

In [3]:
from collections import defaultdict

In [5]:
text = "apple banana apple orange banana apple"
word_count = defaultdict(int)  # int() возвращает 0 по умолчанию

for word in text.split():
    word_count[word] += 1

print(word_count)
print()

data = [("a", 1), ("b", 2), ("a", 3)]
grouped_data = defaultdict(list)

for key, value in data:
    grouped_data[key].append(value)

print(grouped_data)

defaultdict(<class 'int'>, {'apple': 3, 'banana': 2, 'orange': 1})

defaultdict(<class 'list'>, {'a': [1, 3], 'b': [2]})


**Counter** - это подкласс словаря, предназначенный для подсчета хешируемых объектов.

In [8]:
from collections import Counter

In [10]:
text = "abracadabra"
char_count = Counter(text)

print(char_count)

# most_common(n) — возвращает n самых часто встречающихся элементов.
# elements() — возвращает итератор по элементам, повторяющимся столько раз, сколько их количество.

print(char_count.most_common(2))
print(list(char_count.elements()))

Counter({'a': 5, 'b': 2, 'r': 2, 'c': 1, 'd': 1})
[('a', 5), ('b', 2)]
['a', 'a', 'a', 'a', 'a', 'b', 'b', 'r', 'r', 'c', 'd']


**namedtuple** - это подкласс кортежа с именованными полями. Это делает код более читаемым, так как доступ к элементам осуществляется по именам, а не по индексам.

In [13]:
from collections import namedtuple

In [17]:
Point = namedtuple('Point', ['x', 'y'])
p = Point(10, 20)

print(p.x, p.y, p[0], p[1])

10 20 10 20


#### Практика

In [20]:
# Counter для подсчета частоты слов в тексте. Выведите 3 самых часто встречающихся слова.
text = "apple banana apple orange banana apple mango mango mango"
word_count = Counter(text.split())
print(word_count.most_common(3))

[('apple', 3), ('mango', 3), ('banana', 2)]


In [24]:
# Сгруппировка списка кортежей по первому элементу с использованием defaultdict
data = [("a", 1), ("b", 2), ("a", 3), ("c", 4), ("b", 5)]
grouped_data = defaultdict(list)

for key, value in data:
    grouped_data[key].append(value)

print(grouped_data)

defaultdict(<class 'list'>, {'a': [1, 3], 'b': [2, 5], 'c': [4]})


In [26]:
# Создадим namedtuple для представления книги с полями: title, author, year.
# Создадим несколько экземпляров и выведите их.
Book = namedtuple('Book', ['title', 'author', 'year'])
book1 = Book("1984", "George Orwell", 1949)
book2 = Book("To Kill a Mockingbird", "Harper Lee", 1960)

print(book1)
print(book2.author)

Book(title='1984', author='George Orwell', year=1949)
Harper Lee


### 1.1.6. Написание утилит с использованием декораторов и генераторов

In [30]:
# Декоратор для логирования выполнения функций
def logger(func):
    def wrapper(*args, **kwargs):
        print(f"Вызов функции {func.__name__}  с аргументами {args} и {kwargs}")
        result = func(*args, **kwargs)
        print(f"Функция {func.__name__} завершила выполнение с результатом {result}")
        return result
    return wrapper

@logger
def add(a, b):
    return a + b

@logger
def greet(name):
    return f"Привет, {name}"

print(add(5, 4))
print(greet("Alex"))

Вызов функции add  с аргументами (5, 4) и {}
Функция add завершила выполнение с результатом 9
9
Вызов функции greet  с аргументами ('Alex',) и {}
Функция greet завершила выполнение с результатом Привет, Alex
Привет, Alex


In [34]:
# Декоратор для кэширования результатов функций
def cache(func):
    cached_results = {}
    def wrapper(*args):
        if args in cached_results:
            print("Результат из кэша")
            return cached_results[args]
        result = func(*args)
        cached_results[args] = result
        print("Результат вычислен")
        return result
    return wrapper

@cache
def fibonacci(n):
    if n <= 1:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)

# Пример использования
print(fibonacci(3))

Результат вычислен
Результат вычислен
Результат вычислен
Результат из кэша
Результат вычислен
2


In [42]:
# Генератор для бесконечной последовательности чисел
def infinite_sequence(start):
    current = start
    while True:
        yield current
        current += 1

# Пример использования
gen = infinite_sequence(10)
for _ in range(5):
    print(next(gen))

10
11
12
13
14


[Утилита для анализа текста](https://github.com/AnatolyKuzmin/Python/blob/main/1_Python_и_ООП/text_analysis.py)

Логировать выполнение функций.  
Кэшировать результаты анализа.  
Читать большие текстовые файлы построчно.  
Подсчитывать частоту слов в тексте.  
